# Imports

In [ ]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from collections import defaultdict

import numpy as np
import pandas as pd
from scipy import stats

import common_functions
import utils

# Aim of this notebook  
* Calculate the Estimated Daily Intake values via dust ingestion and dermal absorption based on the wristband and dust measurements

In [ ]:
DUST_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath("dust.parquet.gzip")

SERUM_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath(
    "HBM4EU_E-waste_template_V3_all_data_INTERIM.parquet.gzip"
)

WRISTBAND_DATA_PATH = utils.Configuration.INTERIM_DATA_PATH.joinpath(
    "wristband.parquet.gzip"
)

dust = pd.read_parquet(DUST_DATA_PATH).groupby("group").mean()
serum = pd.read_parquet(SERUM_DATA_PATH)
wristband = pd.read_parquet(WRISTBAND_DATA_PATH)

# Calculate EDI via ingestion


Based on the concentrations detailed, we estimate exposure levels through the ingestion of dust and wristband. The estimated daily intake (EDI) in milligrams per kilogram of body weight per day (mg/kg bw/day) was determined using a methodology based on the general approach described by McGrath et al., 2022. The EDI is calculated using the following formula:

**Dust ingestion (ng kg-bw-1 day-1)** = (Concentration × Ingestion × Fraction x Ba) / Body weight
* Concentration (ng/g) : Concentration of analyte in dust/wristband
* Ingestion (g/day): dust ingestion rates of 0.02 g/day for adults in the 50th percentile exposure scenario, and 0.05 g/day for adults in the 95th percentile exposure scenario (USEPA, 2017a.).
* Fraction :  fraction of time individuals spend at the workplace (0.33, 8 hours of work divided by 24 hours)
* Ba : Theoretical bioavailability (calculated from logKow) (CompTox Chemicals Dashboard(Williams et al., 2017)
* Body weight (kg): body weight of participant


## Calculate Bioavailability (Ba)

In [ ]:
# logKow values come from comptox.epa.gov:
logkow = pd.Series(
    {
        "PCB 28": 5.62,
        "PCB 52": 6.18,
        "PCB 101": 6.10,
        "PCB 118": 7.11,
        "PCB 138": 7.34,
        "PCB 153": 6.56,
        "PCB 180": 7.72,
        "BDE 28": 5.94,
        "BDE 47": 6.81,
        "BDE 99": 7.32,
        "BDE 100": 7.24,
        "BDE 153": 7.90,
        "BDE 154": 7.82,
        "BDE 183": 8.27,
        "BDE 209": 7.74,
    }
)

a = 0.2
b = 0.8

# Based on Christina Christia (2021):
Ba = a + ((b - a) * (8 - logkow) / (8 - 5))
Ba

## Dust

To calculate the 50th percentile exposure scenario, we use dust ingestion rates of 20 mg/day. To get the unit of **mg/kg/day** we will use 0.02 g/day intake and divide the final values by 1000, since the original dust measurements are in µg/g dust.

In [ ]:
def calculate_EDI_via_ingestion_dust(ingestion_rate):
    """
    ingestion_rate: 0.02 g/day represents the 50th percentile exposure scenario for adults
                    0.05 g/day represents the 95th percentile exposure scenario for adults
    """
    return (
        serum.query("main_category == 'Worker'")[["companyID", "weight"]]
        .pipe(
            lambda df: df.assign(
                **{
                    col
                    + "": lambda df, col=col: df.companyID.map(
                        dict(
                            zip(dust.index, dust[col] * 1000)
                        )  # multiplied by 1000 to convert µg/g dust to ng/g dust
                    )
                    for col in dust.columns
                }
            )
        )
        .pipe(
            lambda df: df.assign(
                **{
                    col
                    + "": lambda df, col=col: (df[col] * ingestion_rate * (8 / 24))
                    / df["weight"]
                    for col in df.loc[:, "PCB 28":"BDE 209"].columns
                }
            )
        )
        .loc[:, "PCB 28":]
        # .div(1000)
        .mul(Ba)
    )

### 50th percentile


In [ ]:
# EDI values per compound in mg/kg/day

dust_EDI_p50 = calculate_EDI_via_ingestion_dust(ingestion_rate=0.02)
dust_EDI_p50

In [ ]:
# percentiles per compound in mg/kg/day
(dust_EDI_p50.describe().transpose().loc[:, ["count", "25%", "50%", "75%"]])

### 95th percentile
To calculate the 95th percentile exposure scenario, we use dust ingestion rates of 60 mg/day, that is 0.06 g/day.

In [ ]:
# EDI values per compound in mg/kg/day

dust_EDI_p95 = calculate_EDI_via_ingestion_dust(0.06)

In [ ]:
# percentiles per compound in mg/kg/day
(
    dust_EDI_p95.loc[:, "PCB 28":]
    .describe()
    .transpose()
    .loc[:, ["count", "25%", "50%", "75%"]]
    .to_clipboard()
)

### Sum EDI values for PCBs and PBDEs


In [ ]:
for class_ in ["PCB", "BDE"]:
    print(
        f"Median for 50th perentile exposure scenario {class_}: {dust_EDI_p50.filter(like=class_).sum(axis=1).median()}"
    )

In [ ]:
for class_ in ["PCB", "BDE"]:
    print(
        f"Median for 95th perentile exposure scenario {class_}: {dust_EDI_p95.filter(like=class_).sum(axis=1).median()}"
    )

## Based on the wristband measurements
### 50th percentile

To get the unit of mg/kg/day we will use 0.02 g/day intake and divide the final values by 1000_000, since the original wristband measurements are in ng/g wristband.

In [ ]:
def calculate_EDI_via_ingestion_WB(ingestion_rate):
    """
    ingestion_rate: 0.02 g/day represents the 50th percentile exposure scenario for adults
                    0.05 g/day represents the 95th percentile exposure scenario for adults
    """
    return (
        serum[["weight"]]
        .merge(wristband, left_index=True, right_on="ID")
        .drop(columns=["ID", "main_category", "company_ID"])
        .pipe(
            lambda df: df.assign(
                **{
                    col
                    + "_EDI_p50": lambda df, col=col: (
                        df[col] * ingestion_rate * (8 / 24)
                    )
                    / df["weight"]
                    for col in df.loc[:, "PCB 101":"BDE 209"].columns
                }
            )
        )
        .loc[:, "PCB 101_EDI_p50":]
        .rename(columns=lambda x: x.replace("_EDI_p50", ""))
        .mul(Ba)
    )

In [ ]:
wristband_EDI_p50 = calculate_EDI_via_ingestion_WB(0.02)

(wristband_EDI_p50.describe().transpose().loc[:, ["count", "25%", "50%", "75%"]])

### 95th percentile

To get the unit of mg/kg/day we will use 0.06 g/day intake and divide the final values by 1000_000, since the original wristband measurements are in ng/g wristband.

In [ ]:
wristband_EDI_p95 = calculate_EDI_via_ingestion_WB(0.06)
wristband_EDI_p95

# Calculate EDI via dermal intake

Based on doi.org/10.1016/j.chemosphere.2009.02.068 and doi.org/10.1021/acs.est.3c06174: 
EDI(dermal) = C x BSA x SAS x AF x IEF / BW x 1000, where:  

* C = concentration (ng/g dry wt)
* BSA = body surface area (cm2/day) == 4615
* SAS = soil adhered to skin (mg/cm2) == 0.096
* AF = fraction of PBDE absorbed in the skin == 0.03 , fraction of PCB absorbed in the skin == 0.14 (DOI: 10.1006/rtph.2002.1539)
* IEF = indoor exposure fraction (hours spent over a day in an indoor environment) == 0.34
* BW = body weight (k g)


## Based on dust measurements

In [ ]:
# doi.org/10.1016/j.envint.2019.04.056 :
BSA = 2430
SAS = 0.01
PBDE_AF = 0.03
PCB_AF = 0.14
IEF = 8 / 24

In [ ]:
def calculate_dust_dermal_exposure(AF, contaminant_class):
    """
    Calculates the expected dermal exposure from dust.
    Input values:
    AF : float = fraction of analyte absorbed in the skin, it is 0.03 for PBDEs and 0.14 for PCBs
    contaminant_class : str = analyte class to filter by, if PCBs contaminant_class == 'PCB' else 'BDE'
    """
    return (
        serum.query("main_category == 'Worker'")[["companyID", "weight"]]
        .pipe(
            lambda df: df.assign(
                **{
                    col
                    + "": lambda df, col=col: df.companyID.map(
                        dict(
                            zip(dust.index, dust[col] * 1000)
                        )  # we multiply here by 1000 because the original unit is µg/g and we need ng/g here
                    )
                    for col in dust.columns
                }
            )
        )
        .pipe(
            lambda df: df.assign(
                **{
                    col
                    + "": lambda df, col=col: (df[col] * BSA * SAS * AF * IEF)
                    / (df["weight"] * 1000)
                    for col in df.columns[df.columns.str.contains(contaminant_class)]
                }
            )
        )
        .loc[:, lambda df: df.columns.str.contains(contaminant_class)]
    )

In [ ]:
dust_dermal_intake_PCB = calculate_dust_dermal_exposure(PCB_AF, "PCB")
dust_dermal_intake_PCB.sum(axis=1).describe(percentiles=[0.5, 0.95])

In [ ]:
dust_dermal_intake_PBDE = calculate_dust_dermal_exposure(PBDE_AF, "BDE")
dust_dermal_intake_PBDE.sum(axis=1).describe(percentiles=[0.5, 0.95])

# Calculate EDI based on doi.org/10.1016/j.envint.2019.04.056

**Dermal absorption (pg kg-bw-1 day-1)** = Dust concentration (pg g-1) * BSA (cm2) * DAS (mg cm-2) * Fa (unitless) * exposure duration (day) / body weight (kg).  

* BSA = Body surface area for only head and hand exposure (BSA)(US EPA, 2011), 2430 cm2 (male), 2030 cm2 (female)
* Dust adhered to skin (DAS)(US EPA, 2011) 0.01 mg cm-2
* Fractions of target compound absorbed by skin (Fa)(US EPA, 2011) BDE-47 : 0.0285, BDE-99 : 0.0196, BDE-183 : 0.0005, BDE-209 : Not Absorbed
* Exposure duration	8 hours or 0.33 day
* Body weight (US EPA, 2011) : 73 kg (female), 85 kg (male)

**Dust ingestion (pg kg-bw-1 day-1)** = Dust concentration (pg g-1) * dust ingestion rate (g day-1) * exposure duration (day) / body weight (kg).
* Dust ingestion rate (US EPA, 2011) : 0.06 g day-1
* Exposure duration	8 hours or 0.33 day
* Body weight (US EPA, 2011) : 73 kg (female), 85 kg (male)
